In [9]:
# 读取所有 CA 原子（记录残基号和坐标）
ca_atoms = []
with open('step3_input.pdb', 'r') as f:
    for line in f:
        if line.startswith(('ATOM', 'HETATM')):
            atom_name = line[12:16].strip()
            if atom_name == 'CA':
                res_seq_str = line[22:26].strip()
                try:
                    res_seq = int(res_seq_str)
                except ValueError:
                    continue  # 跳过无法解析的残基号
                atom_serial = int(line[6:11].strip())
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                ca_atoms.append((atom_serial, res_seq, x, y, z))

# 计算接触对
contact_pairs = []
threshold = 8.0  # 距离阈值（Å）

for i in range(len(ca_atoms)):
    for j in range(i + 1, len(ca_atoms)):
        a1 = ca_atoms[i]
        a2 = ca_atoms[j]

        # 条件1：至少一个残基在6-36范围内
        in_range1 = (6 <= a1[1] <= 36)
        in_range2 = (6 <= a2[1] <= 36)
        if not (in_range1 or in_range2):
            continue

        # 条件2：排除两个残基都在6-36的情况
        if in_range1 and in_range2:
            continue

        # 条件3：排除同一链相邻残基（间隔<4）
        if abs(a1[1] - a2[1]) < 4:
            continue

        # 计算距离
        distance = ((a1[2] - a2[2]) ** 2 +
                    (a1[3] - a2[3]) ** 2 +
                    (a1[4] - a2[4]) ** 2) ** 0.5
        if distance <= threshold:
            contact_pairs.append((a1[0], a2[0]))

# 输出接触对信息
print("CA原子接触对（原子序号）：")
for pair in contact_pairs:
    print(f"{pair[0]} {pair[1]}")

# 生成 plumed.dat 文件
with open('plumed.dat', 'w') as f:
    f.write("cmap: CONTACTMAP ...\n")

    weight = 1.0 / len(contact_pairs) if contact_pairs else 0
    for idx, (a1_serial, a2_serial) in enumerate(contact_pairs, start=1):
        # 查找对应原子信息用于计算距离
        atom1 = next(atom for atom in ca_atoms if atom[0] == a1_serial)
        atom2 = next(atom for atom in ca_atoms if atom[0] == a2_serial)
        ref_dist = (((atom1[2] - atom2[2]) ** 2 +
                    (atom1[3] - atom2[3]) ** 2 +
                    (atom1[4] - atom2[4]) ** 2) ** 0.5)/10  # 转换为nm

        f.write(f"   ATOMS{idx}={a1_serial},{a2_serial} SWITCH{idx}={{Q R_0=0.01 BETA=50.0 LAMBDA=1.5 REF={ref_dist:.4f}}}  WEIGHT{idx}={weight:.6f} \n")

    f.write("   SUM\n")
    f.write("...\n")
    f.write("PRINT ARG=cmap FILE=colvar\n")


CA原子接触对（原子序号）：
13750 16071
13750 16121
13767 16071
13817 16071
13817 16101
13817 16121
13885 15807
13885 15821
13902 15807
13933 15807
13953 15796
13953 15807
14065 15759
14065 15776
14089 15703
